### Блок импорта библиотек

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression #
from sklearn.svm import SVR
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split #модель разбиения данных на тест и обучение
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, scale #Модель нормализации данных
%pylab inline
%matplotlib notebook
#pd.set_option('display.max_rows', 500)

Populating the interactive namespace from numpy and matplotlib


### Исследование данных

In [2]:
#Загрузка данных
data=pd.read_csv('./train.csv',  keep_default_na=True)

In [3]:
data.describe()   #есть ли еще функции, выводящие сводную статистику?

,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",...,"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
count,3.865000e+03,3864.000000,3864.000000,3837.000000,3837.000000,3732.000000,3502.000000,3474.000000,3474.000000,3432.000000,...,916.000000,915.000000,914.000000,893.000000,892.000000,912.000000,941.000000,874.000000,844.000000,3865.000000
mean,1.000709e+06,0.872479,76.798111,31633.240872,0.825154,14.723296,10.283959,0.804246,0.745900,135.129178,...,0.823561,0.844209,1.008465,121.754797,120.873428,21.252922,6.443023,22.261474,0.600733,0.082773
std,6.945463e+03,0.162367,10.461654,18736.378445,0.194055,3.612546,3.319948,0.176242,0.199795,52.449535,...,0.185780,0.159041,0.087299,46.269362,46.795666,14.258986,4.804873,34.342797,0.205785,0.063415
min,9.900250e+05,0.227890,34.244062,-126.906522,0.292001,3.799663,1.928166,0.273684,0.189874,9.925906,...,0.305733,0.369519,0.465177,23.224603,16.215151,-76.741414,-1.192011,-735.186886,0.089092,0.013036
25%,9.950060e+05,0.763027,69.961449,20169.118912,0.677131,12.592467,7.654169,0.671862,0.597746,92.262724,...,0.690707,0.731264,0.965800,84.090816,82.232550,15.001028,4.113472,17.227899,0.430332,0.047889
50%,1.000000e+06,0.907359,78.995101,26600.768195,0.827300,14.942913,10.385465,0.824758,0.761255,135.914318,...,0.835410,0.862773,1.029947,120.069916,121.057923,22.182571,5.309497,24.472557,0.624640,0.057820
75%,1.006009e+06,0.992760,84.558971,36898.631754,0.970295,17.123797,12.884752,0.939043,0.893505,175.301993,...,0.970365,0.961369,1.068481,158.579644,157.815625,29.134738,6.814577,31.748295,0.767404,0.087389
max,1.015056e+06,1.246908,100.210053,151072.683156,1.361883,26.955944,19.057648,1.232814,1.269625,278.786613,...,1.237661,1.182746,1.181230,232.720847,233.915373,61.903641,36.538462,95.941245,1.098439,0.683813


In [4]:
data.count(axis=1)#статистика по выбранным колонкам

0       38
1       53
2       53
3       24
4       46
        ..
3860    80
3861    80
3862    80
3863    79
3864    79
Length: 3865, dtype: int64

In [5]:
data.info()   #проверка типов признаков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 80 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   galactic year                                                                            3865 non-null   int64  
 1   galaxy                                                                                   3865 non-null   object 
 2   existence expectancy index                                                               3864 non-null   float64
 3   existence expectancy at birth                                                            3864 non-null   float64
 4   Gross income per capita                                                                  3837 non-null   float64
 5   Income Index                                                  

In [6]:
df = data.dropna(axis=1, thresh=0.3*data.count().max())
df = df.iloc[:,df.columns != 'galaxy']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 48 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   galactic year                                                                            3865 non-null   int64  
 1   existence expectancy index                                                               3864 non-null   float64
 2   existence expectancy at birth                                                            3864 non-null   float64
 3   Gross income per capita                                                                  3837 non-null   float64
 4   Income Index                                                                             3837 non-null   float64
 5   Expected years of education (galactic years)                  

In [7]:
data['galaxy'].value_counts()

Andromeda XXII[57]                   26
Tucana Dwarf                         26
Andromeda XXIV                       26
Andromeda XIX[60]                    26
Andromeda XII                        26
                                     ..
Pegasus Dwarf Irregular (DDO 216)    17
Aquarius Dwarf Galaxy (DDO 210)      17
Andromeda X                          17
Andromeda XXI[57]                    17
NGC 5253                              1
Name: galaxy, Length: 181, dtype: int64

In [8]:
# Желтые ячейки - отсутствующие данные NaN - yellow cells
plt.figure(figsize=(20,20))
ax=sns.heatmap(df.isnull(), cmap='viridis', robust=True, cbar=False ) 
ax.set(xlabel='Признаки', ylabel='Кол-во записей в образце данных')
plt.show()

<IPython.core.display.Javascript object>

In [9]:
plt.figure(figsize=(12,10))
bx = sns.heatmap(data.iloc[:,data.columns != 1].corr(), yticklabels=True, xticklabels=True, linewidths=.1, annot=True)
plt.show()

<IPython.core.display.Javascript object>

In [10]:
plt.figure(figsize=(12,10))
bx = sns.heatmap(abs(df.corr())>0.8, yticklabels=True, xticklabels=True, linewidths=.1, annot=True)
plt.show()

<IPython.core.display.Javascript object>

In [11]:
df.fillna(df.median(), inplace=True)  #замена пропусков на матожидание по столбцу
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 48 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   galactic year                                                                            3865 non-null   int64  
 1   existence expectancy index                                                               3865 non-null   float64
 2   existence expectancy at birth                                                            3865 non-null   float64
 3   Gross income per capita                                                                  3865 non-null   float64
 4   Income Index                                                                             3865 non-null   float64
 5   Expected years of education (galactic years)                  

In [ ]:
#поработать с выбросами признаков f15-f18, f27, f29: расчитать межквартильное расстояние и удалить значения, в него непопадающие
#P = np.percentile(dX[["f15","f16","f17","f18","f27", "f29"]], [25, 75])
#new_dX = dX[(dX[["f15","f16","f17","f18","f27", "f29"]] > P[0]) & (dX[["f15","f16","f17","f18","f27", "f29"]] < P[1])]
#new_dX
filt_df=dX[["f15","f16","f17","f18","f27", "f29"]]
quant_df = dX.quantile([0.10, 0.85])
filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[0.10,x.name]) & 
                                    (x < quant_df.loc[0.85,x.name])], axis=0)
filt_df.iloc[:,:]


In [ ]:
dX[["f15","f16","f17","f18","f27", "f29"]]=filt_df[["f15","f16","f17","f18","f27", "f29"]]
dX.dropna(inplace=True)
dX = dX.reset_index()
dX = dX.drop(['index'], axis =1)
dX

In [ ]:
"""Выделим вещественные признаки"""
target_col=['y']
timest=['ff1','ff2']
float_col=[col for col in dX.columns if (dX[col].dtypes=='float' or dX[col].dtypes=='int32') and col not in target_col+timest]
dX[float_col]

In [13]:
X = df.iloc[:,df.columns != 47]

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
scaler = MinMaxScaler() 
X_scaled = scaler.fit_transform(principalDf) 
principalDf = pd.DataFrame(data = X_scaled, columns = ['principal component 1', 'principal component 2'])
principalDf

,principal component 1,principal component 2
0,0.131693,0.282473
1,0.154968,0.304608
2,0.032515,0.331530
3,0.142300,0.341375
4,0.516013,0.562435
...,...,...
3860,0.175549,0.304111
3861,0.219930,0.429335
3862,0.230574,0.472768
3863,0.149991,0.350995


In [ ]:
# вывести % объяснения дисперсии 2 главными компонентами

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('principal component 1', fontsize = 15)
ax.set_ylabel('principal component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = df['y']
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [15]:
X_train, X_test, y_train, y_test=train_test_split(principalDf, df['y'], test_size=0.3)

In [16]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1)

In [17]:
svr_poly.fit(X_train, y_train.values.flatten())
#svr_prediction_proba = svr_rbf.predict_proba(X_test)[:, 1]
svr_poly.score(X_test, y_test)

0.25749507356593826

In [ ]:
lr=LogisticRegression(C=0.01) #, penalty='l2')
lr.fit(X_train, y_train.values.flatten())  #предсказать значения целевой переменной
lr_prediction_proba = lr.predict_proba(X_test)[:, 1] #выдать «степень уверенности» в ответе (вероятность)
lr.score(X_test, y_test)

In [ ]:
# случайный лес
rf_clf = RandomForestClassifier(n_estimators=20)
rf_clf.fit(X_train, y_train.values.flatten())
rf_prediction_proba = rf_clf.predict_proba(X_test)[:, 1]
rf_clf.score(X_test, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [ ]:
#наивный байесовский классификатор с гауссовой ядерной функцией
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train.values.flatten())
nb_prediction_proba = nb_clf.predict_proba(X_test)[:, 1]
nb_clf.score(X_test, y_test)

In [ ]:
#наивный байесовский классификатор с полиномиальной функцией
# for classification with discrete features
clf = MultinomialNB()
clf.fit(X_train, y_train.values.flatten()) # такая форма вызова использована, чтобы убрать предупреждение о несоответствии форматов
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.predict(X_test)
clf_prediction_proba = clf.predict_proba(X_test)[:, 1]
clf.score(X_test, y_test)

#поясните ошибку

In [ ]:
#наивный байесовский классификатор с функцией Бернулли 
# BernoulliNB is designed for binary/boolean features
clfB = BernoulliNB()
clfB.fit(X_train, y_train.values.flatten())  
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
clfB.predict(X_test)
clfB_prediction_proba = clfB.predict_proba(X_test)[:, 1]
clfB.score(X_test, y_test)

In [ ]:
def roc_curve_and_score(y_test, pred_proba):
    fpr, tpr, _ = roc_curve(y_test.ravel(), pred_proba.ravel())
    roc_auc = roc_auc_score(y_test.ravel(), pred_proba.ravel())
    return fpr, tpr, roc_auc


plt.figure(figsize=(8, 6))
matplotlib.rcParams.update({'font.size': 14})
plt.grid()
fpr, tpr, roc_auc = roc_curve_and_score(y_test.values.flatten(), rf_prediction_proba) #случайный лес
plt.plot(fpr, tpr, color='darkorange', lw=2,
         label='ROC AUC={0:.3f}'.format(roc_auc))
fpr, tpr, roc_auc = roc_curve_and_score(y_test.values.flatten(), nb_prediction_proba) # наивный байес-гауссиан
plt.plot(fpr, tpr, color='blue', lw=2,
         label='ROC AUC={0:.3f}'.format(roc_auc))
fpr, tpr, roc_auc = roc_curve_and_score(y_test.values.flatten(), lr_prediction_proba) # логистическая регрессия
plt.plot(fpr, tpr, color='green', lw=2,
         label='ROC AUC={0:.3f}'.format(roc_auc))
fpr, tpr, roc_auc = roc_curve_and_score(y_test.values.flatten(), clfB_prediction_proba) #наивный байес-Бернулли
plt.plot(fpr, tpr, color='yellow', lw=2,
         label='ROC AUC={0:.3f}'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.legend(loc="lower right")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.show()

#заработает, когда повысите score 